In [8]:
import numpy as np
import pandas as pd

# RANDOM TIME DATA
date_rng = pd.date_range(start='1/8/2021', end='1/10/2021', freq='s')
df = pd.DataFrame(date_rng, columns=['timestamp'])

np.random.seed(34)
df['value'] = np.random.randint(50, 250, size=(len(date_rng)))
df = df.sample(frac=0.5, random_state=42).sort_values(by=['timestamp'])
df.to_csv('pressure.csv', index=False)

In [11]:
import csv
rdr = csv.reader(open("thermometer.csv"))
rdr

In [5]:
import random
import json
from time import sleep,time
from kafka import KafkaProducer

bootstrap_servers = ['localhost:19092']  # replace with your broker address
topic_name = 'ac_service'

producer = KafkaProducer(
    bootstrap_servers=bootstrap_servers,
    value_serializer=lambda m: json.dumps(m).encode('ascii'))

while True:
    data = {
        'timestamp': int(time()),
        'value': random.uniform(0, 100)
    }
    producer.send(topic_name, value=data)
    sleep(3)  # wait for 1 second before sending the next data point
    # break

KeyboardInterrupt: 

In [ ]:
import argparse
import json
import sys
import time
import socket
from kafka import KafkaConsumer, errors


def msg_process(msg):
    # Print the current time and the message.
    time_start = time.strftime("%Y-%m-%d %H:%M:%S")
    val = msg.value()
    dval = json.loads(val)
    print(time_start, dval)


parser = argparse.ArgumentParser(description=__doc__)
parser.add_argument('topic', type=str, help='Name of the Kafka topic to stream.')

args = parser.parse_args()

conf = {'bootstrap.servers': 'localhost:9092',
            'default.topic.config': {'auto.offset.reset': 'smallest'},
            'group.id': socket.gethostname()}

consumer = KafkaConsumer('dummy_topic',bootstrap_servers='localhost:9092',
                        value_deserializer=lambda m:json.loads(m.decode('ascii')))

try:
    while True:
        msg = consumer.poll(1)
        if msg is None:
            continue

        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event
                sys.stderr.write('%% %s [%d] reached end at offset %d\n' %
                                    (msg.topic(), msg.partition(), msg.offset()))
            elif msg.error().code() == KafkaError.UNKNOWN_TOPIC_OR_PART:
                sys.stderr.write('Topic unknown, creating %s topic\n' %
                                    (args.topic))
            elif msg.error():
                raise KafkaException(msg.error())
        else:
            msg_process(msg)

except KeyboardInterrupt:
    pass

finally:
    # Close down consumer to commit final offsets.
    consumer.close()